<img src="img/logo.svg" align="right" style="background:black;height:50px; padding-left:5px;padding-right:5px;padding-bottom:0px;">

# Distributed, Advanced

In the previous chapter, we saw how to configure and use bodo inside a 
**Jupyter notebook** using **IPython parallel** (and **MPI** under the hood).

In [1]:
import ipyparallel as ipp

c = ipp.Client(profile="mpi")
view = c[:]
view.activate()
view.block = True

import os

view["cwd"] = os.getcwd()
%px cd $cwd

[stdout:0] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:1] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:2] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks
[stdout:3] /home/xmn/dev/bodoai/bodo-benchmarks/notebooks


In this chapter, we will check how to debug and diagnosticate the distribution found by **bodo**.

## Distributed Diagnostics¶

If you want to understand how a **bodo** compiled function is distributing internally the process,
you can set the environment variable `BODO_DISTRIBUTED_DIAGNOSTICS=1` or you can call
`distributed_diagnostics()` on the compiled function.

For the examples here, we will measure the execution time inside the function to avoid
the time for the function compilation (it maybe would add some extra unnecessary time
to the function).

So, lets start loading the same data we used in the previous chapter:

In [2]:
%%px
import time

import bodo
import pandas as pd
import numpy as np

@bodo.jit
def read_flight_csv():
    t0 = time.time()
    df = pd.read_csv(
        'data/nycflights/',
        parse_dates={'Date': [0, 1, 2]},
        # it needs all fields here
        dtype={
            'Year': np.int16,
            'Month': np.int8,
            'DayofMonth': np.int8,
            'DayOfWeek': np.int8,
            'DepTime': np.float32,
            'CRSDepTime': np.float32,
            'ArrTime': np.float32,
            'CRSArrTime': np.float32,
            'UniqueCarrier': str,
            'FlightNum': np.int16,
            'TailNum': str,
            'ActualElapsedTime': np.float32,
            'CRSElapsedTime': np.float32,
            'AirTime': np.float32,
            'ArrDelay': np.float32,
            'DepDelay': np.float32,
            'Origin': str,
            'Dest': str,
            'Distance': np.float32,
            'TaxiIn': np.float32,
            'TaxiOut': np.float32,
            'Cancelled': np.bool_,
            'Diverted': np.bool_,
        }
    )
    t1 = time.time()
    print(t1 - t0, 's')
    return df


df = read_flight_csv()


[stdout:0] 4.598684072494507 s


[stderr:0] 
/home/xmn/.conda/envs/bodoai-nb/lib/python3.8/site-packages/bodo/transforms/distributed_analysis.py:182: BodoWarning: No parallelism found for function 'read_flight_csv'. This could be due to unsupported usage. See distributed diagnostics for more information.
  warnings.warn(BodoWarning(


Now, lets create a simple function using this data and we will call 
`distributed_diagnostics()` to check the internal configuration of the function.

In [3]:
%%px

@bodo.jit
def avg_distance_by_year(df):
    t0 = time.time()
    avg_distance = df.groupby('Year')['Distance'].mean()
    t1 = time.time()
    print(t1 - t0, 's')
    return avg_distance

result = avg_distance_by_year(df)
print(result)

[stdout:0] 
0.056852102279663086 s
Year
1990     815.490565
1991     819.447871
1992     837.750969
1993     862.320965
1994     860.493017
1995     950.021784
1996     965.890125
1997     983.892431
1998    1018.454424
1999    1032.901512
Name: Distance, dtype: float64
[stdout:1] 
Year
1990     815.490565
1991     819.447871
1992     837.750969
1993     862.320965
1994     860.493017
1995     950.021784
1996     965.890125
1997     983.892431
1998    1018.454424
1999    1032.901512
Name: Distance, dtype: float64
[stdout:2] 
Year
1990     815.490565
1991     819.447871
1992     837.750969
1993     862.320965
1994     860.493017
1995     950.021784
1996     965.890125
1997     983.892431
1998    1018.454424
1999    1032.901512
Name: Distance, dtype: float64
[stdout:3] 
Year
1990     815.490565
1991     819.447871
1992     837.750969
1993     862.320965
1994     860.493017
1995     950.021784
1996     965.890125
1997     983.892431
1998    1018.454424
1999    1032.901512
Name: Distance, 

[stderr:0] 
/home/xmn/.conda/envs/bodoai-nb/lib/python3.8/site-packages/bodo/transforms/distributed_analysis.py:182: BodoWarning: No parallelism found for function 'avg_distance_by_year'. This could be due to unsupported usage. See distributed diagnostics for more information.
  warnings.warn(BodoWarning(


In [4]:
%%px

avg_distance_by_year.distributed_diagnostics()

[stdout:0] 
Distributed diagnostics for function avg_distance_by_year, <ipython-input-47-cdec4a558198> (1)

Data distributions:
   df                        REP
   $14call_method.6.12173    REP
   $14call_method.6.12182    REP
   Distance.12185            REP
   Year.12184                REP
   $cyumk.12242.12282        REP
   $I.12196.12283            REP
   $16call_method.7.12205    REP
   $avg_distance.12284       REP
   $52return_value.21        REP

Parfor distributions:
No parfors to distribute.

Distributed listing for function avg_distance_by_year, <ipython-input-47-cdec4a558198> (1)
------------------------------------------------------------| parfor_id/variable: distribution
@bodo.jit                                                   | 
def avg_distance_by_year(df):                               | 
    t0 = time.time()----------------------------------------| df: REP
    avg_distance = df.groupby('Year')['Distance'].mean()----| $14call_method.6.12173: REP, $14call_method.6.12

As can be observed, the process were replicated to the all cores availables. 
When the function is called it raises a warning ( 
`BodoWarning: No parallelism found for function 'avg_distance_by_year`) and we could check this
information using `distributed_diagnostics`, which all the variables are marked as replicated.

We can try to use the *distributed* parameter to help to for the distribution for a variable.
Lets try to apply it to the previous function to see if it improves the performance:

In [5]:
%%px

@bodo.jit(distributed=['df', 'avg_distance'])
def avg_distance_by_year(df):
    t0 = time.time()
    avg_distance = df.groupby('Year')['Distance'].mean()
    t1 = time.time()
    print(t1 - t0, 's')
    return avg_distance


result = avg_distance_by_year(df)
print(result)

[stdout:0] 
0.05691814422607422 s
Year
1996    965.890125
1997    983.892431
Name: Distance, dtype: float64
[stdout:1] 
Year
1991    819.447871
1992    837.750969
Name: Distance, dtype: float64
[stdout:2] 
Year
1990     815.490565
1994     860.493017
1995     950.021784
1998    1018.454424
1999    1032.901512
Name: Distance, dtype: float64
[stdout:3] 
Year
1993    862.320965
Name: Distance, dtype: float64


In [6]:
%%px

avg_distance_by_year.distributed_diagnostics()

[stdout:0] 
Distributed diagnostics for function avg_distance_by_year, <ipython-input-49-f1175f063965> (1)

Data distributions:
   df                          1D_Block_Var
   $14call_method.6.12492      1D_Block_Var
   $14call_method.6.12501      1D_Block_Var
   Distance.12504              1D_Block_Var
   Year.12503                  1D_Block_Var
   $cyumk.12561.12603          1D_Block_Var
   $I.12515.12604              1D_Block_Var
   $16call_method.7.12524      1D_Block_Var
   $avg_distance.12605         1D_Block_Var
   $cgibeiklu.12480.12607      1D_Block_Var
   distributed_return.12483    1D_Block_Var

Parfor distributions:
No parfors to distribute.

Distributed listing for function avg_distance_by_year, <ipython-input-49-f1175f063965> (1)
------------------------------------------------------------| parfor_id/variable: distribution
@bodo.jit(distributed=['df', 'avg_distance'])               | 
def avg_distance_by_year(df):                               | 
    t0 = time.time()------

In this example using **distributed** parameter, we can observe that the result were split across the 
processors and no variable was replicated, with no difference between both execution time (**~0.06 s**).

## Some questions to consider

- Why using **distributed** parameter didn't improve the execution time?
- Would the **distributed** parameter usage have a better the performance for a largest dataset?

## References

For more information check the [bodo documentation page](https://docs.bodo.ai/latest/source/user_guide.html#distributed-diagnostics)